# Cleaning Movie Budget Data Set

In [3]:
import sqlite3
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle

csv_file_path = '/Users/jessiefreelander/Flatiron_labs/Git_lab_Phase Project/Phase-2-Project/Data/tn.movie_budgets.csv'

data = pd.read_csv(csv_file_path, sep=',')

data.head(5)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Renaming Columns

In [4]:
data = data.rename(columns = { 'release_date':'Release Date', 'movie':'Movie', 'production_budget' : 'Production Budget', 'domestic_gross': 'Domestic Gross', 'worldwide_gross':'World Wide Gross'})
data.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


## Cleaning NaN and 0 values

In [5]:
print(data.isna().any())

id                   False
Release Date         False
Movie                False
Production Budget    False
Domestic Gross       False
World Wide Gross     False
dtype: bool


In [6]:
(data==0).any()

id                   False
Release Date         False
Movie                False
Production Budget    False
Domestic Gross       False
World Wide Gross     False
dtype: bool

In [7]:
Wzero_drop = data[data['World Wide Gross'] == 0].index
data = data.drop(index = Wzero_drop)
data = data.reset_index(drop=True)
Dzero_drop = data[data['Domestic Gross'] == 0].index
data = data.drop(index = Dzero_drop)
data = data.reset_index(drop=True)
data.head()

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Normalizing Data

In [8]:
data['Production Budget'] = data['Production Budget'].str.replace('$','')
data['Production Budget'] = data['Production Budget'].str.replace(',','')
data['Production Budget'] = data['Production Budget'].astype(int)
data.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross
0,1,"Dec 18, 2009",Avatar,425000000,"$760,507,625","$2,776,345,279"


In [9]:
data['Domestic Gross'] = data['Domestic Gross'].str.replace('$','')
data['Domestic Gross'] = data['Domestic Gross'].str.replace(',','')
data['Domestic Gross'] = data['Domestic Gross'].astype(int)
data.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,"$2,776,345,279"


In [10]:
data['World Wide Gross'] = data['World Wide Gross'].str.replace('$', '')
data['World Wide Gross'] = data['World Wide Gross'].str.replace(',', '')
data['World Wide Gross'] = data['World Wide Gross'].astype(int)
data.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279


## Profitability Ratio's

In [11]:
data['Domestic Profit Ratio'] = data['Domestic Gross'] / data['Production Budget']
data['World Wide Profit Ratio'] = data['World Wide Gross'] / data['Production Budget']
data.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross,Domestic Profit Ratio,World Wide Profit Ratio
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,1.78943,6.532577


# Exploring and Cleaning im.db Data Set

In [12]:
db_path = '/Users/jessiefreelander/Flatiron_labs/Git_lab_Phase Project/Phase-2-Project/Data/im.db'

conn = sqlite3.connect(db_path)
cur = conn.cursor()


In [23]:
cur.execute("""
        SELECT name FROM sqlite_master 
        Where type = 'table'
        """)
tables = cur.fetchall()
tables

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [41]:
cur.execute("""
            PRAGMA table_info(persons)
            """)
director_columns_info = cur.fetchall()
director_columns_info

[(0, 'person_id', 'TEXT', 0, None, 0),
 (1, 'primary_name', 'TEXT', 0, None, 0),
 (2, 'birth_year', 'REAL', 0, None, 0),
 (3, 'death_year', 'REAL', 0, None, 0),
 (4, 'primary_profession', 'TEXT', 0, None, 0)]

In [52]:
cur.execute(""" SELECT person_id, primary_name FROM persons
            LIMIT 1
            """)
person_rows = cur.fetchall()
person_rows

[('nm0061671', 'Mary Ellen Bauder')]

In [43]:
cur.execute("""
        SELECT * FROM directors
        LIMIT 1
        """)
director_rows = cur.fetchall()
director_rows

[('tt0285252', 'nm0899854')]

In [35]:
cur.execute("""
            PRAGMA table_info(movie_basics)
            """)
columns_info = cur.fetchall()
columns_info

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'primary_title', 'TEXT', 0, None, 0),
 (2, 'original_title', 'TEXT', 0, None, 0),
 (3, 'start_year', 'INTEGER', 0, None, 0),
 (4, 'runtime_minutes', 'REAL', 0, None, 0),
 (5, 'genres', 'TEXT', 0, None, 0)]

In [39]:
cur.execute("""
        SELECT * FROM movie_basics
        LIMIT 1
        """)
rows = cur.fetchall()
rows

[('tt0063540', 'Sunghursh', 'Sunghursh', 2013, 175.0, 'Action,Crime,Drama')]

## Joined Data set into a Dataframe

In [69]:
query = ("""
    SELECT p.primary_name AS Director, 
       mb.genres AS Genres, 
       mb.start_year AS Release_Year, 
       mb.original_title AS Movie
    FROM directors AS d
    JOIN movie_basics AS mb ON d.movie_id = mb.movie_id
    JOIN persons AS p ON d.person_id = p.person_id
        """)

In [70]:
df = pd.read_sql_query(query, conn)
df

,Director,Genres,Release_Year,Movie
0,Tony Vitale,Comedy,2012,Life's a Beach
1,Bill Haley,Drama,2012,Steve Phoenix: The Untold Story
2,Jay Chandrasekhar,Comedy,2012,The Babymakers
3,Jay Chandrasekhar,Comedy,2012,The Babymakers
4,Albert Pyun,Thriller,2010,Bulletface
...,...,...,...,...
291166,Daysi Burbano,Documentary,2018,Madre Luna
291167,Bernard Lessa,"Drama,Mystery",2017,The woman and the river
291168,Tate Nova,"Documentary,History,News",2018,Stone Mountain Georgia 08.28.18
291169,Courtney Faye Powell,"Documentary,History,News",2018,Stone Mountain Georgia 08.28.18


## Normalizing both data sets

In [71]:
data['Movie'] = data['Movie'].str.strip()
df['Movie'] = df['Movie'].str.strip()

In [72]:
data['Movie'] = data['Movie'].str.lower()
df['Movie'] = df['Movie'].str.lower()

# Merged

In [73]:
dataset = pd.merge(data, df, on = 'Movie', how = 'inner')

## Cleaning Merged Data set

In [74]:
cleaned_dataset = dataset.drop_duplicates(subset=['Movie'], keep='first')
cleaned_dataset.head(1)

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross,Domestic Profit Ratio,World Wide Profit Ratio,Director,Genres,Release_Year
0,2,"May 20, 2011",pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,0.587101,2.546673,Rob Marshall,"Action,Adventure,Fantasy",2011


# Cleaned Data Set

In [75]:
cleaned_dataset

,id,Release Date,Movie,Production Budget,Domestic Gross,World Wide Gross,Domestic Profit Ratio,World Wide Profit Ratio,Director,Genres,Release_Year
0,2,"May 20, 2011",pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,0.587101,2.546673,Rob Marshall,"Action,Adventure,Fantasy",2011
5,3,"Jun 7, 2019",dark phoenix,350000000,42762350,149762350,0.122178,0.427892,Simon Kinberg,"Action,Adventure,Sci-Fi",2019
11,4,"May 1, 2015",avengers: age of ultron,330600000,459005868,1403013963,1.388403,4.243841,Joss Whedon,"Action,Adventure,Sci-Fi",2015
16,7,"Apr 27, 2018",avengers: infinity war,300000000,678815482,2048134200,2.262718,6.827114,Anthony Russo,"Action,Adventure,Sci-Fi",2018
40,9,"Nov 17, 2017",justice league,300000000,229024295,655945209,0.763414,2.186484,Zack Snyder,"Action,Adventure,Fantasy",2017
...,...,...,...,...,...,...,...,...,...,...,...
8403,68,"Jul 6, 2001",cure,10000,94596,94596,9.459600,9.459600,Bill Yip,Drama,2011
8405,70,"Apr 1, 1996",bang,10000,527,527,0.052700,0.052700,Zarina Tadjibaeva,None,2015
8409,73,"Jan 13, 2012",newlyweds,9000,4584,4584,0.509333,0.509333,Edward Burns,"Comedy,Drama",2011
8410,78,"Dec 31, 2018",red 11,7000,0,0,0.000000,0.000000,Robert Rodriguez,"Horror,Sci-Fi,Thriller",2019


In [76]:
cleaned_dataset.to_csv('cleaned_dataset.csv', index = False)